In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd


In [6]:
unzipped_path = 'D:\\New folder\\unzipped'

In [7]:
with open(unzipped_path + '\\candidates.csv', 'r') as f:
    candidates = f.readlines()
candidates[:10]

['seriesuid,coordX,coordY,coordZ,class\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,53.21,-244.41,-245.17,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.66,-121.8,-286.62,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-33.66,-72.75,-308.41,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-32.25,-85.36,-362.51,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-26.65,-203.07,-165.07,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-74.99,-114.79,-311.92,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-16.14,-248.61,-239.55,0\n',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,135.89,-141.41,-252.2,0\n']

In [8]:
len(candidates)

551066

In [9]:
df_candidates = pd.read_csv(unzipped_path + '\\candidates.csv')

In [10]:
df_candidates[:10]

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.080000,-67.850000,-311.920000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.210000,-244.410000,-245.170000,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.660000,-121.800000,-286.620000,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.660000,-72.750000,-308.410000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.250000,-85.360000,-362.510000,0
5,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-26.650000,-203.070000,-165.070000,0
6,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-74.990000,-114.790000,-311.920000,0
7,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-16.140000,-248.610000,-239.550000,0
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,135.890000,-141.410000,-252.200000,0
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,90.102285,-68.430847,-218.243396,0


In [11]:
(df_candidates['class'] == 1).sum()

1351

In [12]:
df_candidates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551065 entries, 0 to 551064
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   seriesuid  551065 non-null  object 
 1   coordX     551065 non-null  float64
 2   coordY     551065 non-null  float64
 3   coordZ     551065 non-null  float64
 4   class      551065 non-null  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 21.0+ MB


In [13]:
df_annotations = pd.read_csv(unzipped_path + '\\annotations.csv')

In [14]:
df_annotations[:10]

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
5,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,90.931713,149.027266,-426.544715,18.208570
6,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,89.540769,196.405159,-515.073322,16.381276
7,1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028...,81.509646,54.957219,-150.346423,10.362321
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...,105.055792,19.825260,-91.247251,21.089619
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...,-124.834262,127.247155,-473.064479,10.465854


In [15]:
df_annotations.shape

(1186, 5)

In [16]:
df_annotations.drop('diameter_mm', axis=1).iloc[0] == df_candidates.drop('class', axis=1).iloc[0]

seriesuid     True
coordX       False
coordY       False
coordZ       False
Name: 0, dtype: bool

In [17]:
from collections import namedtuple


CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

In [35]:
import glob


mhd_list = glob.glob(unzipped_path + '/subset*/subset*/*.mhd')
mhd_list[-10:]

['D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.603126300703296693942875967838.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.712472578497712558367294720243.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.733642690503782454656013446707.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.747803439040091794717626507402.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.765459236550358748053283544075.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.765930210026773090100532964804.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.771741891125176943862272696845.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.5.2.1.6279.6001.855232435861303786204450738044.mhd',
 'D:\\New folder\\unzipped\\subset9\\subset9\\1.3.6.1.4.1.14519.

In [79]:
import csv
import functools
import glob
import os


@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob(unzipped_path + '/subset*/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    diameter_dict = {}
    with open(unzipped_path + '/annotations.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

    candidateInfo_list = []
    with open(unzipped_path + '/candidates.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                    if delta_mm > annotationDiameter_mm / 4:
                        break
                else:
                    candidateDiameter_mm = annotationDiameter_mm
                    break

            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool,
                candidateDiameter_mm,
                series_uid,
                candidateCenter_xyz,
            ))

    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [80]:
import csv


diameter_dict = {}
with open(unzipped_path + '/annotations.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationDiameter_mm = float(row[4])
        diameter_dict.setdefault(series_uid, []).append(
            (annotationCenter_xyz, annotationDiameter_mm)
        )

In [81]:
candidateInfo_list = getCandidateInfoList()

In [49]:
len(candidateInfo_list)

551065

In [56]:
candidateInfo_list[:10]

[CandidateInfoTuple(isNodule_bool=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(67.61451718, 85.02525992, -109.8084416)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(47.90350511, 37.60442008, -99.93417567)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(44.19, 37.79, -107.01)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(40.69, 32.19, -97.15)),
 CandidateInfoTuple(isNodule_bool=True, diameter_mm=27.44242293, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.943403138251347598519939390311', center_xyz=(-45.29440163, 74.86925386, -97.52812481)),
 CandidateInfoTuple(isNodule_bool=True, diam

In [58]:
next(c for c in candidateInfo_list if c.diameter_mm == 0 and not c.isNodule_bool)

CandidateInfoTuple(isNodule_bool=False, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335', center_xyz=(107.65, 7.8, -205.1))

In [82]:
next((key, value) for key, value in diameter_dict.items() if len(value) == 9)


('1.3.6.1.4.1.14519.5.2.1.6279.6001.195557219224169985110295082004',
 [((105.0927476, -12.11034538, -163.8563959), 6.67566416),
  ((-65.24188146, -2.009534031, -154.5988093), 5.611381537),
  ((95.50326213, 50.83309312, -98.17735226), 9.697334131),
  ((-44.35081815, -2.878997069, -80.20624421), 6.734930528),
  ((-36.79299198, 30.7687084, -78.63004564), 7.21357337),
  ((114.1812202, 39.58850481, -77.01551257), 6.554327379),
  ((101.203838, 25.22046529, -133.8132453), 6.269450237),
  ((99.7584658, 42.83369719, -142.7906833), 6.39203287),
  ((76.62464946, -2.190466163, -170.1541606), 5.512269698)])

In [26]:
len(diameter_dict.items())


601

In [33]:
df_annotations.shape[0] - df_annotations['seriesuid'].duplicated().sum()

601

In [39]:
df_candidates['seriesuid'].duplicated().sum()

550177

In [86]:
import SimpleITK as sitk
mhd_path = glob.glob(
    unzipped_path + '/subset*/subset*/{}.mhd'.format(series_uid)
)[0]
print(mhd_path)
ct_mhd = sitk.ReadImage(mhd_path)
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

D:\New folder\unzipped\subset8\subset8\1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335.mhd


In [85]:
ct_a.shape

(285, 512, 512)

In [ ]:
ct_a.clip(-1000, 1000, ct_a)

In [89]:
mask = (ct_a > -1000) & (ct_a < 1000)
ct_a_masked = ct_a[mask]
ct_a_masked.shape, 285*512*512

((47330995,), 74711040)

In [88]:
mask.shape

(285, 512, 512)

In [92]:
import collections


IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = (direction_a @ (cri_a * vxSize_a)) + origin_a
    return XyzTuple(*coords_xyz)
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

In [94]:
ct_mhd.GetOrigin(), ct_mhd.GetSpacing(), ct_mhd.GetDirection()

((-155.199997, -142.199997, -316.609985),
 (0.5859379768371582, 0.5859379768371582, 1.25),
 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0))

In [96]:
torch.tensor(ct_mhd.GetDirection()).reshape((3, 3))

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])